In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from valuation.config import RAW_DATA_DIR, SALES_DATA_FILENAME, CATEGORY_DATA_FILENAME, STORE_DATA_FILENAME, SAME_STORE_SALES_DATA_FILENAME, TRAIN_DATA_FILENAME, VALIDATION_DATA_FILENAME, TEST_DATA_FILENAME, CONFIG_CATEGORY_FILEPATH, ConfigReader

from valuation.utils.io import IOService
from valuation.utils.data import DataFramePartitioner
from valuation.dataset import Dominicks
from valuation.utils.print import Printer

# Dominick's Finer Foods Sales Dataset

The Dominick's Finer Foods (DFF) dataset, provided by the James M. Kilts Center at the University of Chicago Booth School of Business, comprises sales transactions across 28 distinct categorize from the 100-store retail chain. Spanning nearly eight years from September 1989 to May 1997, these data constitute transactions at the Uniform Product Code (UPC) level. 

***

## Key Data Fields

* **STORE**: A unique numeric identifier for each retail location.
* **UPC**: The Uniform Product Code, a unique identifier for each distinct product.
* **WEEK**: A numeric value representing the week of the transaction.
* **MOVE**: The total number of individual units sold for a given UPC in a specific store and week.
* **QTY**: The number of items in a promotional bundle (e.g., `3` for a 3-pack). This value is `1` for individually sold items.
* **PRICE**: The shelf price for the bundle or individual item. Revenue is calculated as **`(PRICE * MOVE) / QTY`** to account for bundled sales.
* **PROFIT**: The **gross margin percentage** for the product. A value of `25.3` corresponds to a 25.3% margin.
* **SALE**: A flag indicating a promotion ('B', 'C', 'S'). This flag is noted by the data providers as being inconsistently applied.
* **OK**: A data quality flag where a value of `1` indicates the record is considered valid for analysis.

## Dataset Overview

In [3]:
# Obtain categories and filenames from config
config_reader = ConfigReader()
category_filenames = config_reader.read(CONFIG_CATEGORY_FILEPATH)

# Obtain an overview of the dataset
ds = Dominicks(category_filenames=category_filenames)
ds.info()

,filename,category,stores,weeks,filesize,memory_usage,n_rows,n_columns
0,wana.zip,Analgesics,93,393,31702949,1663426316,7339217,11
1,wbat.zip,Bath Soap,93,266,6098969,372502226,1644557,11
2,wber.zip,Beer,89,303,16620652,911764716,3990672,11
3,wbjc.zip,Bottled Juices,93,393,40492112,1421596058,6222806,11
4,wcer.zip,Cereals,93,367,42402094,1498663916,6602582,11
5,wche.zip,Cheeses-,93,393,63750169,2155065570,9427395,11
6,wcig.zip,Cigarettes,93,399,21041520,1219993032,5398197,11
7,wcoo.zip,Cookies,93,389,62130356,3062750404,13447807,11
8,wcra.zip,Crackers,93,381,17981446,828371296,3624688,11
9,wcso.zip,Canned Soup,93,379,43140921,1597261704,7011243,11


## Data Preprocessing
The data preprocessing stage transforms the raw, transaction-level records into a series of analysis-ready datasets. These files serve distinct purposes, from model training to strategic performance analysis.

***

### Sales Dataset
* **`sales_data.csv`**: Contains cleaned sales data aggregated weekly by store and category. Revenue and gross profit are calculated for each record and the week start and end dates are added.

### Modeling Datasets
The primary dataset is partitioned chronologically for model development.

* **`train.csv`**: The **Training Set** contains the first 280 weeks (~70%) of the data, and are used to train forecasting models. All performance analysis is derived from this subset.
* **`validation.csv`**: The **Validation Set** comprises the next 60 weeks of data (~15%) and are used for hyperparameter tuning.
* **`test.csv`**: The **Test Set is the hold-test set containing the final 60 weeks of data. This dataset is set aside for unbiased model evaluation.

### Performance Analysis Datasets
Summary datasets are derived from the training set to support strategic analysis.

* **`same_store_sales_growth.csv`**: Contains the aggregated year-over-year Same-Store Sales (SSS) growth for the company.
* **`store_performance.csv`**: Details store-level metrics from the final year of the training data, including year-over-year sales growth, total gross profit, and gross margin percentage.
* **`category_performance.csv`**: Details category-level metrics, structured identically to the store performance file.

## Sales Dataset
The 28 category-level files are now processed and concatenated into a single, aggregated dataset.

In [ ]:
# Obtain categories and filenames from config

from valuation.prep import SalesDataPrep

# Instantiate the config reader and read the category filenames
config_reader = ConfigReader()
category_filenames = config_reader.read(CONFIG_CATEGORY_FILEPATH)

# Instantiate the sales data processor
processor = SalesDataPrep()

# Run the processor pipeline
processor.prepare(category_filenames=category_filenames, force=True)

## Modeling Datasets
For model development, the preprocessed dataset is partitioned chronologically into training, validation, and test subsets. The split is based on a 70/15/15 division of the total weeks in the dataset, which simulates a real-world forecasting scenario by training on past data to predict future outcomes.

In [ ]:
# Instantiate Services
partitioner = DataFramePartitioner()
printer = Printer()

# Load Data
sales_data = IOService.read(SALES_DATA_FILEPATH)

# Split Data
splits = partitioner.split_by_proportion_of_values(df=sales_data, val_col='week', train_size=0.7, val_size=0.15)

# Save Split Data
IOService.write(data=splits['data'].get("train",None), filepath=TRAIN_DATA_FILEPATH)
IOService.write(data=splits['data'].get("validation",None), filepath=VALIDATION_DATA_FILEPATH)
IOService.write(data=splits['data'].get("test",None), filepath=TEST_DATA_FILEPATH)

# Print Split Metadata
printer.print_dict(data=splits['meta'], title="Dataset Split Metadata")